In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import numpy as np
from scipy.stats import multivariate_normal as mvnorm
from scipy.stats import uniform, invgamma, bernoulli, poisson, norm
import statsmodels.api as sm
import pandas as pd
from preprocessing import prepare_data, prepare_data_no_standardizing, MAR_data_deletion
from model_code import Gibbs_MH

In [18]:
df = pd.read_csv('student-mat.csv', sep=",")
features = ["age", "sex", "failures", "higher", "Medu", 
            "absences", "G2", "G3"]
df = df[features]
df.head()

,age,sex,failures,higher,Medu,absences,G2,G3
0,18,F,0,yes,4,6,6,6
1,17,F,0,yes,1,4,5,6
2,15,F,3,yes,1,10,8,10
3,15,F,0,yes,4,2,14,15
4,16,F,0,yes,3,4,10,10


In [19]:
X_df, y_df = prepare_data_no_standardizing(df)
X_df.head()

,age,failures,Medu,absences,G2,sex_M,higher_yes,intercept
0,18,0,4,6,6,0,1,1
1,17,0,1,4,5,0,1,1
2,15,3,1,10,8,0,1,1
3,15,0,4,2,14,0,1,1
4,16,0,3,4,10,0,1,1


In [20]:
X_df_missing = MAR_data_deletion(X_df, 0.05, 0.05, 'higher_yes', 'absences')
X_df_missing

,age,failures,Medu,absences,G2,sex_M,higher_yes,intercept
0,18,0,4,6.0,6,0,1.0,1
1,17,0,1,4.0,5,0,1.0,1
2,15,3,1,10.0,8,0,1.0,1
3,15,0,4,2.0,14,0,1.0,1
4,16,0,3,4.0,10,0,1.0,1
...,...,...,...,...,...,...,...,...
390,20,2,2,NaN,9,1,1.0,1
391,17,0,3,3.0,16,1,1.0,1
392,21,3,1,3.0,8,1,1.0,1
393,18,0,3,0.0,12,1,1.0,1


In [21]:
X = X_df.to_numpy()
y = y_df.to_numpy()
B = 2000
n = len(y)
higher_yes_col = 6
absences_col = 3
age_col = 0
tau = 0.05

In [23]:
(betas, sigmas2, higher_yes_sim, absences_sim, alphas0, alphas1, gammas0, gammas1,
 accepts_alpha0, accepts_alpha1, accepts_gamma0, accepts_gamma0) = Gibbs_MH(X, y, B, n, higher_yes_col, absences_col, age_col, tau)

715.4363563280127
800.1442090094042
4922.73814660127
3874.805344427542
18089.47095168737
14484.888172908864
39793.09728197285
112576.57278447461
129733.31558844168
391667.8199185827
541174.8576892639
459580.9010535423
1379731.3818153362
793031.4216146949
6652577.252537922
3436712.1748693194
26628925.402525317
7782997.050859122
116443218.8751217
26093132.954823293
367372674.98072433
85416084.04374704
957983703.7766037
192235689.45781657
2086896486.0462766
310192745.13860023
10705675199.219702
681570708.0165762
37657752151.205
1590035979.0977407
237062016786.57864
9861446808.370073
1761926592898.7273
34864481231.59178
5282357518736.238
234884466858.18738
10351559516874.68
1313543692038.4792
32417393184990.54
1959990971372.8677
133623398844584.69
4537117111976.896
205161047904693.3
13192147001780.488
555424773231775.5
53596660061255.77
1997880507977558.5
66576142426236.56
1.9129938346246704e+16
435965408983020.06
2.04276702860311e+16
1602660188266446.0
1.0546056345857411e+17
3676555438309

5.334475495172701e+105
5.1143805313077736e+104
2.8705738076248336e+106
3.1362763356633315e+105
1.2316224255763829e+107
1.7419031063593268e+106
6.830544891135166e+107
6.887169985024479e+106
1.875512910344083e+108
3.7949697934457915e+107
6.127777799992122e+108
2.0722987782017146e+108
1.2053844012535434e+109
7.76224505700888e+108
3.066840029952096e+109
2.987874383946839e+109
5.013114051494405e+109
3.169859931444008e+109
6.73787503999141e+109
5.692839576665175e+109
2.7701871452984947e+110
2.8097028225023927e+110
1.5251577390220273e+111
1.672617561922243e+111
7.434833806353518e+111
1.1305820117431543e+112
5.905770117177776e+112
7.218620990852856e+112
5.04107553586912e+113
4.9017061011931466e+113
1.6562471964010256e+114
3.533031014414407e+114
2.6752750506516315e+114
7.437031112086649e+114
7.203067693562164e+114
1.4335799683744873e+115
5.7056906782517685e+115
6.221714619298705e+115
3.162679230419636e+116
2.7226881922739423e+116
7.969903946569623e+116
1.3865238377460573e+117
1.9558815968673e+1

3.4688083740756625e+231
4.962162324702235e+231
2.791682143932995e+232
1.8507044546145478e+232
1.6608089182821678e+233
5.044650829871665e+232
9.018393409230773e+233
1.251610065602098e+233
3.42091348565804e+234
4.955651884715656e+233
4.953550823061832e+234
6.748298456838272e+233
2.4621012776964466e+235
3.845302921066819e+234
8.542969228114547e+235
2.984012215072202e+235
1.5972483329845792e+236
8.714793517248275e+235
2.3476699748051795e+236
7.580898929048617e+236
1.628851535391024e+237
5.09188685917182e+237
4.793295726444791e+237
2.0144616449369606e+238
8.187222229299422e+237
1.4508737287498178e+239
4.1866991792641074e+238
4.44340039979371e+239
6.766435320616188e+238
4.621665828204521e+240
3.3552415987201993e+239
1.5705297412780594e+241
7.042218445675386e+239
1.0173407059044351e+242
4.743990843673666e+240
3.3657613042620954e+242
2.4256692082452073e+241
5.859479587825003e+242
6.693426473217589e+241
1.886440090392496e+243
9.815371104305261e+241
1.4146210864163772e+244
3.4599242328223385e+24

ValueError: array must not contain infs or NaNs

printing above the $(y - X.dot(betas[:, i-1])).T.dot(y - X.dot(betas[:, i-1])) / 2$ parameter in the inverse gamma of sigma2. Increases until infinity which causes it to crash. 